## Import Libraries

In [47]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib inline

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential, constraints, layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import SeparableConv2D, Input, Conv2D, Add, BatchNormalization, concatenate, AveragePooling2D, add, MaxPooling2D 
from tensorflow.keras.layers import Conv2DTranspose, Activation, Dropout,UpSampling2D ,ZeroPadding2D, LeakyReLU
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [12]:
reduction_ratio=16

### Xception

In [41]:
def Xception(input_shape=(28, 28, 3)):
    # 28x28x3
    inputs = Input(input_shape)
    
    # 28x28x32
    x = layers.Conv2D(32, (3, 3), padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    # 28x28x64
    x = layers.Conv2D(64, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    # 28x28x128
    residual = layers.Conv2D(128, (1, 1), padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)
    # 28x28x128
    x = layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    # 28x28x128
    x = layers.add([x, residual])
    # 14x14x256
    residual = layers.Conv2D(256, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(56, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    # 14x14x256
    x = layers.MaxPooling2D((3, 3),strides=(2, 2),padding='same')(x)
    x = layers.add([x, residual])
    # 7x7x728
    residual = layers.Conv2D(728, (1, 1),strides=(2, 2), padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    # 7x7x728
    x = layers.MaxPooling2D((3, 3),strides=(2, 2),padding='same')(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, (3, 3),padding='same',use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, (3, 3),padding='same',use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, (3, 3),padding='same',use_bias=False)(x)
        x = layers.BatchNormalization()(x)

        x = layers.add([x, residual])
    
    # 1x1x1024
    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(1024, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    # 1x1x1024
    x = layers.MaxPooling2D((3, 3),strides=(2, 2),padding='same')(x)
    x = layers.add([x, residual])
    x = tf.keras.layers.MaxPool2D((2,2))(x) 
    
    x = layers.Flatten()(x)
    x = layers.Dense(2048, activation='elu', kernel_initializer='he_normal', kernel_constraint=constraints.max_norm(3.))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation='elu', kernel_initializer='he_normal', kernel_constraint=constraints.max_norm(3.))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10 ,activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [42]:
model = Xception()

model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 28, 28, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_106 (Conv2D)             (None, 28, 28, 32)   864         input_20[0][0]                   
__________________________________________________________________________________________________
batch_normalization_630 (BatchN (None, 28, 28, 32)   128         conv2d_106[0][0]                 
__________________________________________________________________________________________________
activation_542 (Activation)     (None, 28, 28, 32)   0           batch_normalization_630[0][0]    
___________________________________________________________________________________________